In [3]:
import h2o
h2o.init()
data = h2o.import_file("towerbridge.csv")

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Picked up _JAVA_OPTIONS: -Djava.net.preferIPv4Stack=trueed mode)
  Starting server from C:\ProgramData\Miniconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\edwarddi\AppData\Local\Temp\tmp1of40oys
  JVM stdout: C:\Users\edwarddi\AppData\Local\Temp\tmp1of40oys\h2o_edwarddi_started_from_python.out
  JVM stderr: C:\Users\edwarddi\AppData\Local\Temp\tmp1of40oys\h2o_edwarddi_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,06 secs
H2O cluster timezone:,Europe/London
H2O data parsing timezone:,UTC
H2O cluster version:,3.20.0.10
H2O cluster version age:,5 days
H2O cluster name:,H2O_from_python_edwarddi_eerjqv
H2O cluster total nodes:,1
H2O cluster free memory:,3.531 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%


### Our Data
To use it with H2O modelling, we've converted our raster image (a grid of colour values) to a list of tuples, mapping the pixel coordinates to the colour values.

In [4]:
data

C1,b,g,r,x,y
0,96,28,0,-0.0233463,-0.0233463
1,94,28,0,-0.0231632,-0.0233463
2,94,28,0,-0.0229801,-0.0233463
3,93,29,0,-0.022797,-0.0233463
4,94,30,0,-0.0226139,-0.0233463
5,95,29,0,-0.0224308,-0.0233463
6,96,29,0,-0.0222477,-0.0233463
7,97,29,0,-0.0220645,-0.0233463
8,99,28,0,-0.0218814,-0.0233463
9,101,28,0,-0.0216983,-0.0233463


In [ ]:
y = "b"
x = ["x","y"]

train, valid, test = data.split_frame([0.75, 0.15])

from h2o.estimators import H2ODeepLearningEstimator
m = H2ODeepLearningEstimator(model_id="DL_defaults", hidden=[10,10,10,10,10,10,10,10,10,10], activation='tanh',epochs=1000)
m.train(x,y,train)
m

deeplearning Model Build progress: |███

## Rendering our results
To see our model's output, we need to feed it the coordinates of the pixels we want rendered

In [57]:
import numpy as np
import pandas as pd
from h2o.frame import H2OFrame
from PIL import Image

IM_SIZE = 256
IM_CHANNELS = 1

def save_pixels(path_to_image_file, image_array, mode):
    im_out = Image.fromarray(image_array, mode )
    im_out.save(path_to_image_file)


# Takes care of clipping, casting to int8, etc.
def save_ndarray(path_to_outfile, x, width = IM_SIZE, height = IM_SIZE, channels = IM_CHANNELS):

    out_arr = np.clip(x, 0, 255)
    
    if channels == 3:
        out_arr = np.reshape(out_arr, (width, height, channels), 1)
    else:
        assert(channels == 1)
        out_arr = np.reshape(out_arr, (width, height), 1)
        
    out_arr = np.rot90(out_arr, k=3)
    out_arr = np.fliplr(out_arr)
    
    if channels == 3:
        save_pixels(path_to_outfile, out_arr.astype(np.int8), 'RGB')
    else:
        save_pixels(path_to_outfile, out_arr.astype(np.int8), 'L')


# Create suitable training matrix
def gen_input_tuples(pixels_width, pixels_height, scale, translate_x, translate_y):

    image_height = pixels_height
    image_width = pixels_width

    # One row per pixel
    X = np.zeros((image_width * image_height, 2))

    # Fill in y values
    X[:,1] = np.repeat(range(0, image_height), image_width, 0)

    # Fill in x values
    X[:,0] = np.tile(range(0, image_width), image_height)

    X = X + np.asarray([0,translate_x])
    X = X + np.asarray([translate_y,0])

    # Normalize X
    X = X - X.mean()
    X = X / X.var()

    X = X * scale

    return (X)
    
pixel_coords = gen_input_tuples(256, 256, 1, 0, 0)
df_pixels_to_render = pd.DataFrame({'x':pixel_coords[:,0], 'y':pixel_coords[:,1]})
h2o_pixels = H2OFrame(df_pixels_to_render)
pixel_intensities = m.predict(h2o_pixels)

save_ndarray("modelled_tower_bridge.png", pixel_intensities.as_data_frame().as_matrix(), IM_SIZE, IM_SIZE, 1)

Parse progress: |█████████████████████████████████████████████████████████| 100%
deeplearning prediction progress: |███████████████████████████████████████| 100%


C:\ProgramData\Miniconda3\lib\site-packages\ipykernel_launcher.py:65: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


![modelled_tower_bridge.png](modelled_tower_bridge.png)

In [48]:
print(m)

Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  DL_defaults


ModelMetricsRegression: deeplearning
** Reported on train data. **

MSE: 2230.507266100853
RMSE: 47.22824648556045
MAE: 32.74329044532223
RMSLE: NaN
Mean Residual Deviance: 2230.507266100853
Scoring History: 


,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_deviance,training_mae,training_r2
,2018-10-23 12:15:18,0.000 sec,None,0.0,0,0.0,nan,nan,nan,nan
,2018-10-23 12:15:25,7.485 sec,6968 obs/sec,1.0,1,49144.0,51.6807577,2670.9007182,40.1757933,0.1369572
,2018-10-23 12:15:31,12.605 sec,12291 obs/sec,3.0,3,147432.0,49.5356531,2453.7809296,33.9030883,0.2071147
,2018-10-23 12:15:36,17.760 sec,17395 obs/sec,6.0,6,294864.0,49.5411756,2454.3280798,33.3520517,0.2069379
,2018-10-23 12:15:42,23.615 sec,21729 obs/sec,10.0,10,491440.0,47.2282465,2230.5072661,32.7432904,0.2792607


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
x,1.0,1.0,0.5760385
y,0.7359949,0.7359949,0.4239615
